In [4]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from datetime import datetime

class MetaAdAnalyzer:
    def __init__(self):
        self.targets = {
            'ROAS': 2.0,
            'CPA': 26,
            'uATC': 16,
            'uCV': 70,
            'uPV': 400
        }
    
    def analyze_daily_data(self, data):
        """
        Analyze daily Meta ad data
        
        Parameters:
        data: DataFrame with columns for date, spend, revenue, purchases, etc.
        """
        results = {}
        
        # Calculate confidence intervals and statistics
        for metric in ['ROAS', 'CPA', 'uATC', 'uCV', 'uPV']:
            if metric in data.columns:
                metric_stats = self._calculate_statistics(data[metric], metric)
                results[metric] = {
                    'current_value': metric_stats['mean'],
                    'confidence_interval': (metric_stats['ci_lower'], metric_stats['ci_upper']),
                    'cv': metric_stats['cv'],
                    'p_value': metric_stats['p_value'],
                    'sample_size': metric_stats['n'],
                    'target': self.targets.get(metric),
                    'status': self._get_status(metric_stats, metric)
                }
        
        return results

    def _calculate_statistics(self, values, metric):
        """Calculate key statistics for a metric"""
        mean = values.mean()
        std = values.std()
        n = len(values)
        
        # Calculate CV
        cv = (std / mean * 100) if mean != 0 else float('inf')
        
        # Calculate confidence interval
        confidence_level = 0.95
        margin_of_error = stats.t.ppf((1 + confidence_level) / 2, n - 1) * (std / np.sqrt(n))
        ci_lower = mean - margin_of_error
        ci_upper = mean + margin_of_error
        
        # Calculate p-value against target
        target = self.targets.get(metric, mean)
        t_stat, p_value = stats.ttest_1samp(values, target)
        
        return {
            'mean': mean,
            'std': std,
            'cv': cv,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper,
            'p_value': p_value,
            'n': n
        }

    def _get_status(self, metric_stats, metric):
        """Determine status based on statistics"""
        if metric_stats['p_value'] > 0.05:
            return "Need more data"
        elif metric_stats['cv'] > 30:
            return "Too volatile"
        elif metric_stats['mean'] >= self.targets.get(metric, 0):
            return "On target"
        else:
            return "Below target"

    def plot_metric(self, data, metric):
        """Plot metric over time with confidence interval"""
        plt.figure(figsize=(10, 6))
        plt.plot(data['date'], data[metric], 'o-', label=f'Daily {metric}')
        
        mean = data[metric].mean()
        ci = stats.t.interval(0.95, len(data)-1, loc=mean, scale=stats.sem(data[metric]))
        
        plt.axhline(y=mean, color='r', linestyle='--', label='Mean')
        plt.axhline(y=ci[0], color='g', linestyle=':', label='95% CI')
        plt.axhline(y=ci[1], color='g', linestyle=':')
        
        plt.title(f'{metric} Over Time')
        plt.xlabel('Date')
        plt.ylabel(metric)
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

# Example usage
if __name__ == "__main__":
    # Example of how to prepare your daily data
    daily_data = {
        'date': pd.date_range(start='2024-01-01', periods=14),
        'ROAS': [2.1, 2.3, 1.9, 2.0, 2.2, 2.4, 2.1, 2.0, 2.3, 2.1, 2.2, 2.0, 2.1, 2.3],
        'CPA': [24, 25, 23, 26, 24, 25, 23, 24, 25, 26, 24, 25, 23, 24],
        'uATC': [18, 16, 17, 15, 19, 16, 17, 18, 15, 16, 17, 18, 16, 17],
        'uCV': [75, 72, 78, 71, 73, 76, 74, 72, 75, 73, 74, 76, 73, 75],
        'uPV': [420, 410, 430, 415, 425, 418, 422, 428, 415, 420, 425, 418, 422, 430]
    }
    df = pd.DataFrame(daily_data)

    # Initialize analyzer
    analyzer = MetaAdAnalyzer()

    # Get analysis results
    results = analyzer.analyze_daily_data(df)

    # Print results
    print("\n=== Meta Ad Performance Analysis ===\n")
    for metric, stats in results.items():
        print(f"\n{metric}:")
        print("-" * 40)
        print(f"Current Value: {stats['current_value']:.2f}")
        print(f"95% CI: ({stats['confidence_interval'][0]:.2f}, {stats['confidence_interval'][1]:.2f})")
        print(f"CV: {stats['cv']:.2f}%")
        print(f"P-value: {stats['p_value']:.4f}")
        print(f"Status: {stats['status']}")



=== Meta Ad Performance Analysis ===


ROAS:
----------------------------------------
Current Value: 2.14
95% CI: (2.06, 2.23)
CV: 6.78%
P-value: 0.0028
Status: On target

CPA:
----------------------------------------
Current Value: 24.36
95% CI: (23.78, 24.94)
CV: 4.14%
P-value: 0.0000
Status: Below target

uATC:
----------------------------------------
Current Value: 16.79
95% CI: (16.10, 17.47)
CV: 7.08%
P-value: 0.0279
Status: On target

uCV:
----------------------------------------
Current Value: 74.07
95% CI: (72.97, 75.17)
CV: 2.57%
P-value: 0.0000
Status: On target

uPV:
----------------------------------------
Current Value: 421.29
95% CI: (417.86, 424.71)
CV: 1.41%
P-value: 0.0000
Status: On target
